In [1]:
def parse(instruction):
    if instruction == "noop":
        return {"type":"noop"}
    return {"type":"addx", "value":int(instruction.split(" ")[-1])}

with open("./input.txt", "r") as file:  
    data = [parse(line) for line in file.read().strip().split("\n")]

# Part 1

In [2]:
class CPU: 
    def __init__(self, register=1, listeners=None):
        self.register  = register
        self.clock     = 0
        self.listeners = listeners or [] 
        
    def tick(self):
        self.clock += 1
        
        for listener in self.listeners: 
            listener(self)
        
    def handle(self, instruction):
        if instruction["type"] == "noop":
            self.tick()
            
        if instruction["type"] == "addx": 
            self.tick()
            self.tick()
            self.register += instruction["value"]
            
    def run(self, instructions):
        for instruction in instructions: 
            self.handle(instruction)
        
def run(data): 
    history = []
    
    # on each tick, record the value of the register
    # and the clock
    machine = CPU(
        listeners = [
            lambda cpu: history.append({
                "register":cpu.register, 
                "clock":cpu.clock,
                "signal":cpu.register * cpu.clock
            })
        ]
    )
    
    # process all the instructions
    machine.run(data)
        
    # snapshots 
    timestamps = [20, 60, 100, 140, 180, 220]
    
    return sum([
        x["signal"] for x in history if x["clock"] in timestamps]
    )
    

    
run(data)  

13720

# Part 2

In [3]:
class CRT: 
    def __init__(self):
        self.screen     = []
        self.position   = 0
        self.cpu = CPU(1, [
            lambda cpu: self.draw(), 
            lambda cpu: self.move()
        ])
        
    @property
    def sprite(self):
        """
        Returns the position of the 3-pixel wide sprite
        """
        return (self.cpu.register - 1, self.cpu.register, self.cpu.register + 1)
        
    def draw(self):
        if self.position in self.sprite: 
            self.screen.append("#")
        else: 
            self.screen.append(".")
    
    def move(self):
        self.position = (self.position + 1) % 40
        
    def display(self): 
        def chunk(iterable, width): 
            for i in range(0, len(self.screen), width):
                yield self.screen[i:i+width]
        
        return "\n".join(["".join(line) for line in (chunk(self.screen, 40))])
        
crt = CRT()
crt.cpu.run(data)
print(crt.display())

####.###..#..#.###..#..#.####..##..#..#.
#....#..#.#..#.#..#.#..#....#.#..#.#..#.
###..###..#..#.#..#.####...#..#....####.
#....#..#.#..#.###..#..#..#...#....#..#.
#....#..#.#..#.#.#..#..#.#....#..#.#..#.
#....###...##..#..#.#..#.####..##..#..#.
